<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/AF2_ESM_Fold_Fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##Clean an AlphaFold2 or ESMFold protein structure for use with MemProtMD

This Notebook uses [PDB2PQR](https://onlinelibrary.wiley.com/doi/10.1002/pro.3280), [available here](https://github.com/Electrostatics/pdb2pqr), followed by energy minimisation using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).



In [ ]:
#@title Initialisation
%%capture
!python3 -m pip install py3dmol
!apt-get update
!apt-get upgrade
!apt-get install gzip
!apt-get install pymol
File_Location = "ESMFold Sequence" #@param ["Upload","AFDB","ESMFold Sequence"]

#@markdown ---
#@markdown #### AlphaFold database:
UniProtID = "P00804" #@param {type:"string"}
#@markdown ---
#@markdown ##### ESMFold:
Sequence_name = 'LspA' #@param {type:"string"}
Method = 'ESMfold' 
Sequence = 'MSQSICSTGLRWLWLVVVVLIIDLGSKYLILQNFALGDTVPLFPSLNLHYARNYGAAFSFLADSGGWQRWFFAGIAIGISVILAVMMYRSKATQKLNNIAYALIIGGALGNLFDRLWHGFVVDMIDFYVGDWHFATFNLADTAICVGAALIVLEGFLPSRAKKQ' #@param {type:"string"}

In [ ]:
#@title Get PDB coordinate file
import os
import sys
import requests
import py3Dmol
from google.colab import files
sys.path.append('/usr/local/lib/python3.7/site-packages/')

os.chdir('/content/')

if File_Location == "Upload":
  upload = files.upload()
  filename = next(iter(upload))
elif File_Location == "AFDB":
  name = str(UniProtID.upper())
  os.system('wget https://alphafold.ebi.ac.uk/files/AF-' + name + '-F1-model_v4.pdb')
  filename = name + '.pdb'
  os.rename('AF-' + name + '-F1-model_v4.pdb',filename)
elif File_Location == "Sequence":
  name = Sequence_name  
  if Method == "Helix":  
    with open('helix.pml','w') as pml:
      pml.write('set retain_order,0\nset secondary_structure,1\nfor aa in "'+ Sequence +'": cmd._alt(str.upper(aa))\nsave '+Sequence_name+'.pdb')
      !pymol -cq helix.pml
  elif Method == "ESMfold":  
    esmfold_api_url = 'https://api.esmatlas.com/foldSequence/v1/pdb/'
    r = requests.post(esmfold_api_url, data=Sequence)
    structure = r.text
    with open("./"+name+".pdb", 'w') as pdb_file:
      pdb_file.write(structure)
  filename = name + '.pdb'

name = os.path.splitext(filename)[0]
working_dir = '/content/' + name + '/'
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + name + '.pdb')
os.chdir(working_dir)
mol1 = open(working_dir + filename, 'r').read()
mview = py3Dmol.view(width=800,height=400) 
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'resn':'DUM'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()


In [ ]:
#@title Install dependencies
%%capture

os.chdir('/content/')

if not os.path.isdir("/content/content/gromacs/"):
  !apt-get update -y
  !python3 -m pip install pdb2pqr
  !python3 -m pip install GromacsWrapper
  !python3 -m pip install MDAnalysis 
  !python3 -m pip install py3dmol
  !wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/master/gromacs.zip
  !unzip -o gromacs.zip
  !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx
  !ln -s /content/content/gromacs/bin/gmx_d /usr/bin/gmx_d

import gromacs
import MDAnalysis
import py3Dmol
import glob
import shutil

In [ ]:
#@title Energy Minimise
%%capture
os.chdir(working_dir)

if not os.path.exists('temp'):
	os.makedirs('temp')
os.chdir('temp')

print("Assessing "+filename+"\n")

os.system('pdb2pqr30 --ff CHARMM  --keep-chain '+ working_dir + filename + ' pqr.pdb')

gromacs.pdb2gmx(f='pqr.pdb',ignh=True,ff='charmm27',water='tip3p',o='conf.pdb')

gromacs.editconf(f='conf.pdb',d=8,c=True,o='conf.pdb')

with open('em.mdp','w') as em:
            em.write('integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001')

gromacs.grompp(f='em.mdp',maxwarn=5,o='em',c='conf.pdb')

gromacs.mdrun(deffnm='em',c='clean.pdb')

gromacs.trjconv(f='clean.pdb',o=working_dir + 'fixed-'+filename,s='em.tpr',input=('system'))

os.chdir('..')

shutil.rmtree('temp', ignore_errors=True)

In [ ]:
#@title Download
mview = py3Dmol.view(width=800,height=400)  
mol1 = open(working_dir + 'fixed-'+filename, 'r').read()
mview.addModel(mol1,'pdb')
mview.setStyle({'cartoon':{'color':'spectrum'}})
mview.setStyle({'atom':'P'},{'sphere':{}})
mview.setBackgroundColor('0xffffff')
mview.zoomTo()
mview.show()
files.download(working_dir + 'fixed-'+filename)